In [ ]:
#default_exp engines.detectron2.predict

# CLI for using Detectron2 models for inference

In [ ]:
# export
from drone_detector.imports import *
from drone_detector.processing.all import *
from drone_detector.metrics import *
from drone_detector.utils import *

import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import cv2
import torch
from shutil import rmtree

from fastcore.foundation import *
from fastcore.script import *

In [ ]:
# export

@call_parse
def predict_bboxes_detectron2(path_to_model_config:Param("Path to pretrained model folder",type=str),
                              path_to_image:Param("Path to image to annotate", type=str),
                              outfile:Param('Path and filename for output raster', type=str),
                              processing_dir:Param("Directory to save the intermediate tiles. Deleted after use", type=str, default='temp'),
                              tile_size:Param("Tile size to use. Default 400x400px tiles", type=int, default=400),
                              tile_overlap:Param("Tile overlap to use. Default 100px", type=int, default=200),
                              use_tta:Param("Use test-time augmentation", store_true),
                              smooth_preds:Param("Run fill_holes and dilate_erode to masks", store_true)
    ):
    "Detect bounding boxes from a new image using a pretrained model"
    
    if os.path.exists(processing_dir): 
        print('Processing folder exists')
        return
    os.makedirs(processing_dir)
    print(f'Reading and tiling {path_to_image} to {tile_size}x{tile_size} tiles with overlap of {tile_overlap}px')
    tiler = Tiler(outpath=processing_dir, gridsize_x=int(tile_size), gridsize_y=int(tile_size), 
                  overlap=(int(tile_overlap), int(tile_overlap)))
    tiler.tile_raster(path_to_image)
    
    # Check whether is possible to use gpu
    device = 'cpu' if not torch.cuda.is_available() else f'cuda:{torch.cuda.current_device()}'
    
        # Loading pretrained model
    print('Loading model')
    
    cfg = get_cfg()
    cfg.merge_from_file(path_to_model_config)
    cfg.DEVICE = device
    
    print('Starting predictions')
    image_files = os.listdir(f'{processing_dir}/raster_tiles')
    if use_tta:
        print('Not yet implemented')
    #else:
    
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
    predictor = DefaultPredictor(cfg)    

    images = []
    preds = []

    for i in tqdm(rangeof(image_files)):
        img = cv2.imread(f'{processing_dir}/raster_tiles/{image_files[i]}')
        pred = (predictor(img[...,::-1]))
        images.append({'file_name': image_files[i],
                       'width': img.shape[0],
                       'height': img.shape[0],
                       'id': i+1})
        preds.append(pred)

    # Mask postprocessing:
    if smooth_preds:
        print('Not yet implemented')
        #preds = fill_holes(preds)
        #preds = dilate_erode(preds)
        
    preds_coco = detectron2_bbox_preds_to_coco_anns(images, preds)
    # TODO fix categories to not be hardcoded    
    preds_coco['categories'] = [
        {'supercategory':'deadwood', 'id':1, 'name': 'uprightwood'},
        {'supercategory':'deadwood', 'id':2, 'name': 'groundwood'},
    ]
    
                         

    # Process preds to shapefiles
    coco_proc = COCOProcessor(data_path=processing_dir, 
                              outpath=processing_dir,
                              coco_info=None, coco_licenses=None,
                              coco_categories=preds_coco['categories'])
    
    
    coco_proc.coco_to_shp(preds_coco, downsample_factor=1)
    
    # Discard all predictions that are not completely within -2m erosion of grid cell
    # TODO add as optional postprocessing step
    
    grid = tiler.grid
    grid = grid.to_crs('EPSG:3067')
    
    for cell in grid.itertuples():
        if not os.path.isfile(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson'): continue
        pred_shp = gpd.read_file(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson')
        orig_crs = pred_shp.crs
        pred_shp = pred_shp.to_crs('EPSG:3067')
        cell_geom = cell.geometry.buffer(-1) # 1 unit erosion (~25px) from each side, hope that crs unit is meters. 
        pred_shp['to_drop'] = pred_shp.apply(lambda row: 0 if row.geometry.intersects(cell_geom) else 1, axis=1)
        pred_shp = pred_shp[pred_shp.to_drop == 0]
        pred_shp = pred_shp.to_crs(orig_crs)
        pred_shp.drop(columns=['to_drop'], inplace=True)
        if use_tta:
            pred_shp = do_nms(pred_shp, 0.1, 'score')
        if len(pred_shp) > 0: pred_shp.to_file(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson')
        else: os.remove(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson')

    # Collate shapefiles
    untile_vector(path_to_targets=f'{processing_dir}/predicted_vectors', outpath=outfile)
    
    print('Removing intermediate files')
    rmtree(processing_dir)
    return

In [ ]:
# export

@call_parse
def predict_instance_masks_detectron2(path_to_model_config:Param("Path to pretrained model folder",type=str),
                                      path_to_image:Param("Path to image to annotate", type=str),
                                      outfile:Param('Path and filename for output raster', type=str),
                                      processing_dir:Param("Directory to save the intermediate tiles. Deleted after use", type=str, default='temp'),
                                      tile_size:Param("Tile size to use. Default 400x400px tiles", type=int, default=400),
                                      tile_overlap:Param("Tile overlap to use. Default 100px", type=int, default=200),
                                      use_tta:Param("Use test-time augmentation", store_true),
                                      smooth_preds:Param("Run fill_holes and dilate_erode to masks", store_true)
    ):
    "Segment instance masks from a new image using a pretrained model"
    
    if os.path.exists(processing_dir): 
        print('Processing folder exists')
        return
    os.makedirs(processing_dir)
    print(f'Reading and tiling {path_to_image} to {tile_size}x{tile_size} tiles with overlap of {tile_overlap}px')
    tiler = Tiler(outpath=processing_dir, gridsize_x=int(tile_size), gridsize_y=int(tile_size), 
                  overlap=(int(tile_overlap), int(tile_overlap)))
    tiler.tile_raster(path_to_image)
    
    # Check whether is possible to use gpu
    device = 'cpu' if not torch.cuda.is_available() else f'cuda:{torch.cuda.current_device()}'
    
        # Loading pretrained model
    print('Loading model')
    
    cfg = get_cfg()
    cfg.merge_from_file(path_to_model_config)
    cfg.DEVICE = device
    
    print('Starting predictions')
    image_files = os.listdir(f'{processing_dir}/raster_tiles')
    if use_tta:
        print('Not yet implemented')
    #else:
    
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
    predictor = DefaultPredictor(cfg)    

    images = []
    preds = []

    for i in tqdm(rangeof(image_files)):
        img = cv2.imread(f'{processing_dir}/raster_tiles/{image_files[i]}')
        pred = (predictor(img[...,::-1]))
        images.append({'file_name': image_files[i],
                       'width': img.shape[0],
                       'height': img.shape[0],
                       'id': i+1})
        preds.append(pred)

    # Mask postprocessing:
    if smooth_preds:
        print('Not yet implemented')
        #preds = fill_holes(preds)
        #preds = dilate_erode(preds)
        
    preds_coco = detectron2_mask_preds_to_coco_anns(images, preds)
    # TODO fix categories to not be hardcoded    
    preds_coco['categories'] = [
        {'supercategory':'deadwood', 'id':1, 'name': 'uprightwood'},
        {'supercategory':'deadwood', 'id':2, 'name': 'groundwood'},
    ]
    
                         

    # Process preds to shapefiles
    coco_proc = COCOProcessor(data_path=processing_dir, 
                              outpath=processing_dir,
                              coco_info=None, coco_licenses=None,
                              coco_categories=preds_coco['categories'])
    
    
    coco_proc.coco_to_shp(preds_coco, downsample_factor=1)
    
    # Discard all predictions that are not completely within -2m erosion of grid cell
    # TODO add as optional postprocessing step
    
    grid = tiler.grid
    grid = grid.to_crs('EPSG:3067')
    
    for cell in grid.itertuples():
        if not os.path.isfile(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson'): continue
        pred_shp = gpd.read_file(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson')
        orig_crs = pred_shp.crs
        pred_shp = pred_shp.to_crs('EPSG:3067')
        cell_geom = cell.geometry.buffer(-1) # 1 unit erosion (~25px) from each side, hope that crs unit is meters. 
        pred_shp['to_drop'] = pred_shp.apply(lambda row: 0 if row.geometry.intersects(cell_geom) else 1, axis=1)
        pred_shp = pred_shp[pred_shp.to_drop == 0]
        pred_shp = pred_shp.to_crs(orig_crs)
        pred_shp.drop(columns=['to_drop'], inplace=True)
        if use_tta:
            pred_shp = do_nms(pred_shp, 0.1, 'score')
        if len(pred_shp) > 0: pred_shp.to_file(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson')
        else: os.remove(f'{processing_dir}/predicted_vectors/{cell.cell}.geojson')

    # Collate shapefiles
    untile_vector(path_to_targets=f'{processing_dir}/predicted_vectors', outpath=outfile)
    
    print('Removing intermediate files')
    rmtree(processing_dir)
    return